## Comparisons of methods for searching FHIR

This GA4GH Search query searches a copy of the Patient resource from the NCPI FHIR Resource created under Project Forge. The attribute we want to query on, ethnicity, is available as an extension so a level of indirection is necessaey to query on that attribute. The value of ethnicity must then be unpacked.

In [6]:
import json

from fasp.search  import DiscoverySearchClient

searchClient = DiscoverySearchClient('https://ga4gh-search-adapter-presto-public.prod.dnastack.com')

query = """select id, patient from kidsfirst.ga4gh_tables.patient 
where json_extract_scalar(patient, '$.extension[0].url') = 
'http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity' 
limit 3"""
#TODO query on the value of ethnicity with AND

res = searchClient.runQuery(query)

for r in res:
    patient = r[1]
    print(patient['id'], patient['gender'])
    for e in patient['extension']:
        print (e['url'])
        print(e['extension'][0]['url'])
        vc = e['extension'][0]['valueCoding']
        print(vc['code'], vc['display'])

_Retrieving the query_
____Page1_______________
____Page2_______________
____Page3_______________
____Page4_______________
____Page5_______________
451133 male
http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity
ombCategory
2186-5 Not Hispanic or Latino
http://hl7.org/fhir/us/core/StructureDefinition/us-core-race
ombCategory
2106-3 White
451134 female
http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity
ombCategory
2135-2 Hispanic or Latino
http://hl7.org/fhir/us/core/StructureDefinition/us-core-race
ombCategory
2106-3 White
451135 female
http://hl7.org/fhir/us/core/StructureDefinition/us-core-ethnicity
ombCategory
2135-2 Hispanic or Latino
http://hl7.org/fhir/us/core/StructureDefinition/us-core-race
ombCategory
2106-3 White


### Perform the same query directly via FHIR
Is it any easier to specify the query and to unpack the results?

Using the NIH Cloud Platform Interoperability (NCPI) FHIR server directly.

Note the file with the cookie for the NCPI FHIR server should contain the following
{"Cookie":"AWSELBAuthSessionCookie-0=your_cookie_here"}

First a basic query to check we can query the FHIR server via the fhir-py library.

In [10]:
from fhirpy import SyncFHIRClient
import os
import json


endpoint = 'https://ncpi-api-fhir-service-dev.kidsfirstdrc.org'
full_cookie_path = os.path.expanduser('~/.keys/ncpi_fhir_cookie.json')
with open(full_cookie_path) as f:
        cookies = json.load(f)

client = SyncFHIRClient(endpoint, extra_headers=cookies)

# Search for patients by gender
resources = client.resources('Patient')  
resources = resources.search(gender='female').limit(1000)
patients = resources.fetch_all()
print("# of patients:{}".format(len(patients)))

# of patients:1074


Working on formulating in fhir-py the query as above

In [8]:
# Search for patients by race
resources = client.resources('Patient')  
resources = resources.search(race='white').limit(1000)
patients = resources.fetch_all()
print("# of patients:{}".format(len(patients)))

# of patients:0
